Importamos dependencias

In [88]:
print("Hola")

Hola


In [89]:
# # Warning control
# import warnings
# warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv

load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew

Definimos el modelo

In [90]:
from crewai import LLM

llm = LLM(
    model="openai/gpt-4o-mini", # call model by provider/model_name
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)

Carga de instrucciones

In [91]:
files = {
    'agents': '../../app/config/hair_diagno_config/agents.yaml',
    'tasks': '../../app/config/hair_diagno_config/tasks.yaml'
}

configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs['agents']
tasks_config = configs['tasks']

Pydantic model for the diagnostic task

In [92]:
from typing import List
from pydantic import BaseModel, Field, field_validator
from enum import Enum

class ToneTemperature(str, Enum):
    WARM = "cálido"
    COLD = "frío"
    NEUTRAL = "neutro"

class DamageLevel(str, Enum):
    LIGHT = "ligero"
    MODERATE = "moderado"
    SEVERE = "severo"

class Porosity(str, Enum):
    LOW = "baja"
    MEDIUM = "media"
    HIGH = "alta"

class HairCharacteristics(BaseModel):
    porosity: Porosity = Field(..., description="Nivel de porosidad del cabello")
    thickness: str = Field(..., description="Grosor del cabello (fino, medio, grueso)")
    density: str = Field(..., description="Densidad del cabello (baja, media, alta)")
    texture: str = Field(..., description="Textura del cabello (liso, ondulado, rizado, etc.)")
    damage: DamageLevel = Field(..., description="Nivel de daño del cabello")

class HairDiagnostic(BaseModel):
    base_tone_height: int = Field(
        ..., 
        description="Altura del tono base en escala internacional (1-10)",
        ge=1, # greater than or equal
        le=10 # less than or equal
    )
    gray_hair_percentage: float = Field(
        ..., 
        description="Porcentaje aproximado de canas presentes",
        ge=0, 
        le=100
    )
    mid_ends_color_description: str = Field(
        ...,
        description="Descripción detallada del color en medios y puntas"
    )
    tone_temperature: ToneTemperature = Field(
        ...,
        description="Temperatura del tono actual (cálido, frío, neutro)"
    )
    tone_description: str = Field(
        ...,
        description="Descripción técnica completa del tono con matices específicos"
    )
    hair_characteristics: HairCharacteristics = Field(
        ...,
        description="Características estructurales del cabello"
    )
    recommendations: List[str] = Field(
        default=[],
        description="Recomendaciones basadas en el diagnóstico"
    )
 
    
    # @field_validator('base_tone_height')
    # def validate_tone_height(cls, v):
    #     if v < 1 or v > 10:
    #         raise ValueError('La altura del tono debe estar entre 1 y 10')
    #     return v

     

IMAGE ANALYZER TOOL

In [93]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
import base64
import os
from openai import OpenAI
from typing import Optional, Type
from PIL import Image
import io

# 1. Definimos un esquema compatible con CrewAI v0.22+
class AnalyzeImageArgs(BaseModel):
    image_path: str = Field(..., description="Ruta al archivo de imagen a analizar")

# 2. Creamos una herramienta compatible con BaseTool
class AnalyzeImageTool(BaseTool):
    name: str = "analyze_image"
    description: str = "Analiza una imagen de cabello utilizando Vision API"
    args_schema: Type[BaseModel] = AnalyzeImageArgs
    
    def _run(self, image_path: str) -> str:
        try:
            # Verificar si la imagen existe
            image_path = os.path.abspath(os.path.expanduser(image_path))
            if not os.path.exists(image_path):
                return f"Error: La imagen no existe en la ruta: {image_path}"
            
            # Inicializar el cliente de OpenAI
            client = OpenAI()
            
            # Detectar si es WEBP y convertir a JPEG si es necesario
            base64_image = ""
            try:
                with Image.open(image_path) as img:
                    # Si es WEBP u otro formato no compatible directamente, convertir a JPEG
                    if img.format != 'JPEG':
                        buffer = io.BytesIO()
                        img = img.convert('RGB')  # Elimina canal alfa si existe
                        img.save(buffer, format='JPEG')
                        buffer.seek(0)
                        base64_image = base64.b64encode(buffer.read()).decode("utf-8")
                    else:
                        # Si ya es JPEG, usar directamente
                        with open(image_path, "rb") as image_file:
                            base64_image = base64.b64encode(image_file.read()).decode("utf-8")
            except Exception as img_error:
                return f"Error al procesar la imagen: {str(img_error)}"
            
            prompt = """Analizar detalladamente la imagen del cabello proporcionada y generar un diagnóstico profesional completo. El análisis debe:
                - Determinar la altura del tono base utilizando la escala internacional de colorimetría (1-10)
                - Calcular el porcentaje aproximado de canas presentes
                - Identificar y describir el color en medios y puntas
                - Proporcionar una descripción técnica del tono actual (cálido, frío, neutro)
                - Analizar las características del cabello (porosity, thickness, density, texture, damage)
                
                Si no puedes procesar la imagen claramente o no contiene cabello visible, NO inventes ningún diagnóstico,
                simplemente indica que no puedes realizar el análisis."""
            
            # Crear la solicitud a la API
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                        ],
                    }
                ],
                max_tokens=800  # Aumentado para dar espacio a respuestas más detalladas
            )
            
            return response.choices[0].message.content
            
        except Exception as e:
            return f"Error al analizar la imagen: {str(e)}"

Agnents & Crews

In [94]:
# Creating Agents
hair_diagno = Agent(
  config=agents_config['hair_diagno'],  
  llm=llm,
  tools=[AnalyzeImageTool()],
  verbose=True
)

hair_diagno_generation = Task(
  config=tasks_config['hair_diagno_generation'],
  agent=hair_diagno,
  output_pydantic= HairDiagnostic
)

crew= Crew(
  agents=[hair_diagno],
  tasks=[hair_diagno_generation],
  verbose=True
)

# Usamos una ruta absoluta para asegurarnos de que se encuentra la imagen
# Alternativamente, puedes probar con una imagen que sepas que existe

image_path = '../../assets/preview.webp'
# Para depuración, verifica la existencia de la imagen antes de ejecutar
print(f"¿La imagen existe? {os.path.exists(image_path)}")

result = crew.kickoff(inputs={'image_path': image_path})
print(result)

Overriding of current TracerProvider is not allowed


¿La imagen existe? True
# Agent: Consultor de Diagnóstico Capilar
## Task: Utiliza la herramienta disponible para generar el diagnostico pasandole como parametro ../../assets/preview.webp. Una vez recibido el diagnostico, estructuralo en el formato indicado en el expected_output. y No sugierasningun diagnostico si no puedes leer, encodear o procesar la imagen


# Agent: Consultor de Diagnóstico Capilar
## Using tool: analyze_image
## Tool Input: 
"{\"image_path\": \"../../assets/preview.webp\"}"
## Tool Output: 
No puedo realizar un análisis detallado del cabello a partir de la imagen proporcionada. Para obtener un diagnóstico preciso, te recomendaría consultar a un profesional cualificado en colorimetría o cuidado capilar. Ellos pueden evaluar adecuadamente las características del cabello y proporcionar recomendaciones basadas en un análisis en persona.


# Agent: Consultor de Diagnóstico Capilar
## Final Answer: 
No puedo realizar un análisis detallado del cabello a partir de la imag